In [1]:
import pandas as pd
import numpy as np

In [2]:
# read csv
df = pd.read_csv("anonymized-curriculum-access.txt", sep=" ", header=None)
# rename columns
df.columns = ["dates", "time", "endpoint", "user_id", "cohort", 'ip']
# finding unique pages 
df.endpoint.unique()

array(['/', 'java-ii', 'java-ii/object-oriented-programming', ...,
       'florence-python-assessment.html', 'javascript-i/dom',
       'appendix/professional-development/post-interview-review-form'],
      dtype=object)